In [2]:
import torch 
import torchvision
import torch.nn as nn # All neural network modules, Liner, Conv2d, BN, LF
import torch.optim as optim # SGD, Adam, etc.
import torch.nn.functional as F # ALL functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our data

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [55]:
# Create a RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # N x time_sequence x features
        # self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Forward Prop
        # out, _ = self.rnn(x, h0)
        out, _ = self.lstm(x, (h0, c0))
        # out = out.reshape(out.shape[0], -1)
        # out = self.fc(out)
        out = self.fc(out[:, -1, :])
        return out

In [5]:
# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform=transforms.ToTensor(),
                               download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False,
                               transform=transforms.ToTensor(),
                               download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                          shuffle=True)

In [44]:
data, tar = next(iter(train_loader))
print(data.squeeze(1).shape)
print(tar)

torch.Size([64, 28, 28])
tensor([9, 3, 2, 4, 4, 9, 6, 1, 7, 6, 5, 6, 3, 2, 9, 3, 8, 9, 1, 0, 0, 2, 2, 6,
        4, 9, 6, 9, 8, 7, 7, 9, 2, 4, 2, 8, 3, 5, 9, 0, 5, 6, 2, 8, 3, 5, 8, 6,
        1, 1, 2, 1, 4, 8, 0, 9, 6, 1, 6, 7, 1, 6, 2, 2])


In [56]:
# Initialize network
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [57]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [58]:
# Train Network
from tqdm import tqdm
for epoch in range(num_epochs):
    for data, targets in tqdm(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()


100%|██████████| 938/938 [00:07<00:00, 126.95it/s]


In [71]:
model.eval()
x, y = next(iter(train_loader))
print(f'x.shape: {x.shape}')
s = model(x.squeeze(1).to(device))
print(f's.shape: {s.shape}')
s.max(1)[1]

x.shape: torch.Size([64, 1, 28, 28])
s.shape: torch.Size([64, 10])


tensor([7, 6, 0, 2, 3, 7, 5, 6, 7, 7, 6, 0, 3, 1, 0, 7, 9, 7, 8, 6, 9, 6, 7, 7,
        6, 1, 8, 1, 9, 5, 0, 5, 4, 8, 4, 8, 2, 9, 6, 1, 0, 1, 7, 4, 5, 3, 8, 8,
        8, 2, 0, 4, 0, 7, 4, 6, 8, 1, 0, 1, 5, 5, 4, 3], device='cuda:0')

In [60]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = round(float(num_correct)/float(num_samples)*100, 2)
        print(f'Got {num_correct} / {num_samples} with accuracy \
              {acc}')
    
    # model.train()

In [61]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58817 / 60000 with accuracy               98.03
Checking accuracy on test data
Got 9768 / 10000 with accuracy               97.68
